In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [205]:
from collections import defaultdict
from PIL import Image
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import enums
from torchvision import transforms
import transformers
import configuration
import customdataset
import feature_helpers
from torchvision import transforms
from matplotlib import pyplot as plt
import torch
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

config = configuration.Configuration()
pose_config = configuration.PoseConfig(config)
face_config = configuration.FaceConfig(config)
# FILES = [(3, 'img_100041.jpg'), (4, 'img_67180.jpg'), (5, 'img_54197.jpg'), (6, 'img_47868.jpg')]
FILES = [(0, 'img_34.jpg')]
TRANSFORMERS = transforms.Compose([transforms.ToTensor(), transformers.PyTorchImageToNPArray()])


In [ ]:
def transform_keypoints(img, key_points):
    PADDING = 20
    grayed = tf.image.rgb_to_grayscale(img, name=None)
    zero_pixels = np.argwhere(grayed == 0)
    border_top = np.min(zero_pixels[:,0])
    border_left  = np.min(zero_pixels[:,1])
    border_bottom = np.max(zero_pixels[:,0])
    border_right = np.max(zero_pixels[:,1])
    
    # Find the crop region, accounting for the white border we just removed.
    positions = np.argwhere(img > 0)
    top = np.min(positions[:,0]) + border_top # points are shifted up by top
    left  = np.min(positions[:,1]) + border_left # points are shifted left by left
    bottom = np.max(positions[:,0]) + border_top
    right = np.max(positions[:,1]) + border_left

    d = max(right - left, bottom - top)
    padding_x = d - (bottom - top)
    padding_y = d - (right - left)
    
    new_keypoints = key_points.copy()
    new_keypoints[:, 0] = PADDING + new_keypoints[:, 0] - (left + border_left)
    new_keypoints[:, 1] = PADDING + new_keypoints[:, 1] - (top + border_top)
    
    # Add padding.
    crop_img = img[top+border_top:bottom, left+border_left:right]
    crop_img = cv2.copyMakeBorder(crop_img, PADDING, PADDING + padding_x, PADDING, padding_y + PADDING, cv2.BORDER_CONSTANT, None, 0)
    width, height = crop_img.shape[0:2]
    aspect_h = 256 / height
    aspect_w = 256 / width

    resized_img = cv2.resize(crop_img, [int(width*aspect_w), int(height*aspect_h)], interpolation=cv2.INTER_AREA)
    resized_w, resized_h = resized_img.shape[0:2]

    x_original_center = (width-1) / 2
    y_original_center = (height-1) / 2

    x_scaled_center = (resized_w-1) / 2
    y_scaled_center = (resized_h-1) / 2
    print(f'height:{height}, width:{width}, aspect_w:{aspect_w}, aspect_h:{aspect_h}, x_original_center:{x_original_center}, y_original_center:{y_original_center}')
    print(f'resized_h:{resized_h}, resized_w:{resized_w}, x_scaled_center:{x_scaled_center}, y_scaled_center:{y_scaled_center}')
    # Subtract the center, scale, and add the "scaled center".
    new_keypoints[:, 0] = (new_keypoints[:, 0] - x_original_center)*aspect_w + x_scaled_center
    new_keypoints[:, 1] = (new_keypoints[:, 1] - y_original_center)*aspect_h + y_scaled_center

    return new_keypoints, resized_img

def load_image(image_path, transformer):
    img = Image.open(image_path)
    if not transformer is None:
        img = transformer(img)
    return img
    
def load_images(poses_folder, label, file_name, transformer):
    original_file_path = f'{config.TRAIN_DATA}/c{label}/{file_name}'
    orig_img = load_image(original_file_path, transformer)

    pose_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_pose.png")}' # img_100041_pose.png
    pose_img = load_image(pose_file_path, transformer)
    
    pose_cropped_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_pose_cropped.png")}' # img_100041_pose_cropped.png
    pose_cropped_img = load_image(pose_cropped_file_path, transformer)
    
    annotated_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_annotated.png")}' # img_100041_annotated.png
    annotated_img = load_image(annotated_file_path, transformer)
    return (orig_img, annotated_img, pose_img, pose_cropped_img)
    
def display_images(axes, orig_img, annotated_img, pose_img, pose_cropped_img):
    axes[0][0].imshow(orig_img)
    axes[0][1].imshow(annotated_img)
    axes[2][0].imshow(pose_img)
    axes[2][1].imshow(pose_cropped_img)
        
def display_keypoints(axes, img, pose_img, pose_cropped_img, keypoints, colors, shapes):
    height, width, channel = img.shape
    aspect_ratio = float(width) / height
    
    num_instances = keypoints.shape[0]
    FIRST_POSE_IDX = 0
    kpts_x = keypoints[0, FIRST_POSE_IDX, :, 1]
    kpts_y = keypoints[0, FIRST_POSE_IDX, :, 0]
    kpts_scores = keypoints[0, FIRST_POSE_IDX, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > pose_config.KEYPOINT_THRESHOLD, :]    

    new_keypoints, new_img = transform_keypoints(pose_img, kpts_absolute_xy)
    print(f'kpts_absolute_xy:{kpts_absolute_xy}')
    print(f'new_keypoints: {new_keypoints}')
    print(f'new_img: {new_img.shape}')
    ax = axes[1]
    im = ax.imshow(img)
    for i, keypoint in enumerate(kpts_absolute_xy):
        if kpts_scores[i] > pose_config.KEYPOINT_THRESHOLD:
            scat = ax.scatter([keypoint[0]], [keypoint[1]], s=15, 
                              color=colors[i], marker=shapes[i], zorder=3, alpha=0.5)
    ax = axes[2]
    im = ax.imshow(new_img)
    for i, keypoint in enumerate(new_keypoints):
        if kpts_scores[i] > pose_config.KEYPOINT_THRESHOLD:
            scat = ax.scatter([int(keypoint[0])], [int(keypoint[1])], s=30, 
                              color=colors[i], marker=shapes[i], zorder=3, alpha=0.5)

            
def normalize_poses_for_label(ax_row, label, file_name, transformer):
    colormap = plt.cm.hsv #nipy_spectral #, Set1,Paired
    colors = [colormap(i) for i in np.linspace(0, 1.0, 17)]
    shapes = ['o', 'v', '^', '<', '>', 
              '1', '2', '4', 's', 'p',
              '*', '+', 'x', 'D', 4,
              5, 6]

    poses_folder = f'{pose_config.FEATURES_FOLDER}/c{label}'

    (orig_img, annotated_img, pose_img, pose_cropped_img) = load_images(poses_folder, label, file_name, transformer)
    
    keypoints_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_keypoints.pt")}' # img_100041_keypoints.pt
    
    keypoints = torch.load(keypoints_file_path) # A numpy array with shape [1, 1, 17, 3] 
                                                # representing the keypoint coordinates and scores
                                                # returned from the MoveNet model.
    disp_img = tf.image.resize_with_pad(orig_img, pose_config.INPUT_SIZE, pose_config.INPUT_SIZE)
    # disp_img = tf.expand_dims(disp_img, axis=0)
    # display_images(axes, orig_img, annotated_img, pose_img, pose_cropped_img)
    ax_row[0].imshow(annotated_img)
    
    display_keypoints(ax_row, disp_img, pose_img, pose_cropped_img, keypoints, colors, shapes)
    
    # fig.tight_layout(pad=0)
    # ax.margins(0)
    # ax.set_yticklabels([])
    # ax.set_xticklabels([])
    # plt.axis('off')
        
def normalize_poses():
    fig, axes = plt.subplots(len(FILES), 3, figsize=[16, 24])
    for axis in axes.flatten():
        axis.set_xticks([])
        axis.set_yticks([])
    if len(FILES) > 1:
        for i, (label, filename) in enumerate(FILES):
            normalize_poses_for_label(axes[i], label, filename, TRANSFORMERS)
    else:
        run(axes, FILES[0][0], FILES[0][1], TRANSFORMERS)

    plt.subplots_adjust(wspace=None, hspace=None)
    plt.tight_layout()
    # plt.subplots_adjust(top=0.88)
    plt.show()
        
# normalize_poses()


In [207]:
def transform_keypoints(img, key_points):
    PADDING = 20
    grayed = tf.image.rgb_to_grayscale(img, name=None)
    zero_pixels = np.argwhere(grayed == 0)
    border_top = np.min(zero_pixels[:,0])
    border_left  = np.min(zero_pixels[:,1])
    border_bottom = np.max(zero_pixels[:,0])
    border_right = np.max(zero_pixels[:,1])
    
    # Find the crop region, accounting for the white border we just removed.
    positions = np.argwhere(img > 0)
    top = np.min(positions[:,0]) + border_top # points are shifted up by top
    left  = np.min(positions[:,1]) + border_left # points are shifted left by left
    bottom = np.max(positions[:,0]) + border_top
    right = np.max(positions[:,1]) + border_left

    d = max(right - left, bottom - top)
    padding_x = d - (bottom - top)
    padding_y = d - (right - left)
    
    new_keypoints = key_points.copy()
    new_keypoints[:, 0] = PADDING + new_keypoints[:, 0] - (left + border_left)
    new_keypoints[:, 1] = PADDING + new_keypoints[:, 1] - (top + border_top)
    
    # Add padding.
    crop_img = img[top+border_top:bottom, left+border_left:right]
    crop_img = cv2.copyMakeBorder(crop_img, PADDING, PADDING + padding_x, PADDING, padding_y + PADDING, cv2.BORDER_CONSTANT, None, 0)
    width, height = crop_img.shape[0:2]
    aspect_h = 256 / height
    aspect_w = 256 / width

    resized_img = cv2.resize(crop_img, [int(width*aspect_w), int(height*aspect_h)], interpolation=cv2.INTER_AREA)
    resized_w, resized_h = resized_img.shape[0:2]

    x_original_center = (width-1) / 2
    y_original_center = (height-1) / 2

    x_scaled_center = (resized_w-1) / 2
    y_scaled_center = (resized_h-1) / 2
    print(f'height:{height}, width:{width}, aspect_w:{aspect_w}, aspect_h:{aspect_h}, x_original_center:{x_original_center}, y_original_center:{y_original_center}')
    print(f'resized_h:{resized_h}, resized_w:{resized_w}, x_scaled_center:{x_scaled_center}, y_scaled_center:{y_scaled_center}')
    # Subtract the center, scale, and add the "scaled center".
    new_keypoints[:, 0] = (new_keypoints[:, 0] - x_original_center)*aspect_w + x_scaled_center
    new_keypoints[:, 1] = (new_keypoints[:, 1] - y_original_center)*aspect_h + y_scaled_center

    return new_keypoints, resized_img

def load_image(image_path, transformer):
    img = Image.open(image_path)
    if not transformer is None:
        img = transformer(img)
    return img
    
def load_images(poses_folder, label, file_name, transformer):
    original_file_path = f'{config.TRAIN_DATA}/c{label}/{file_name}'
    orig_img = load_image(original_file_path, transformer)

    pose_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_pose.png")}' # img_100041_pose.png
    pose_img = load_image(pose_file_path, transformer)
    
    pose_cropped_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_pose_cropped.png")}' # img_100041_pose_cropped.png
    pose_cropped_img = load_image(pose_cropped_file_path, transformer)
    
    annotated_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_annotated.png")}' # img_100041_annotated.png
    annotated_img = load_image(annotated_file_path, transformer)
    return (orig_img, annotated_img, pose_img, pose_cropped_img)
    
def display_images(axes, orig_img, annotated_img, pose_img, pose_cropped_img):
    axes[0][0].imshow(orig_img)
    axes[0][1].imshow(annotated_img)
    axes[2][0].imshow(pose_img)
    axes[2][1].imshow(pose_cropped_img)
        
def display_keypoints(axes, img, pose_img, pose_cropped_img, keypoints, colors, shapes):
    height, width, channel = img.shape
    aspect_ratio = float(width) / height
    
    num_instances = keypoints.shape[0]
    FIRST_POSE_IDX = 0
    kpts_x = keypoints[0, FIRST_POSE_IDX, :, 1]
    kpts_y = keypoints[0, FIRST_POSE_IDX, :, 0]
    kpts_scores = keypoints[0, FIRST_POSE_IDX, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > pose_config.KEYPOINT_THRESHOLD, :]    

    new_keypoints, new_img = transform_keypoints(pose_img, kpts_absolute_xy)
    print(f'kpts_absolute_xy:{kpts_absolute_xy}')
    print(f'new_keypoints: {new_keypoints}')
    print(f'new_img: {new_img.shape}')
    ax = axes[1]
    im = ax.imshow(img)
    for i, keypoint in enumerate(kpts_absolute_xy):
        if kpts_scores[i] > pose_config.KEYPOINT_THRESHOLD:
            scat = ax.scatter([keypoint[0]], [keypoint[1]], s=15, 
                              color=colors[i], marker=shapes[i], zorder=3, alpha=0.5)
    ax = axes[2]
    im = ax.imshow(new_img)
    for i, keypoint in enumerate(new_keypoints):
        if kpts_scores[i] > pose_config.KEYPOINT_THRESHOLD:
            scat = ax.scatter([int(keypoint[0])], [int(keypoint[1])], s=30, 
                              color=colors[i], marker=shapes[i], zorder=3, alpha=0.5)

            
def normalize_poses_for_label(ax_row, label, file_name, transformer):
    colormap = plt.cm.hsv #nipy_spectral #, Set1,Paired
    colors = [colormap(i) for i in np.linspace(0, 1.0, 17)]
    shapes = ['o', 'v', '^', '<', '>', 
              '1', '2', '4', 's', 'p',
              '*', '+', 'x', 'D', 4,
              5, 6]

    poses_folder = f'{pose_config.FEATURES_FOLDER}/c{label}'

    (orig_img, annotated_img, pose_img, pose_cropped_img) = load_images(poses_folder, label, file_name, transformer)
    
    keypoints_file_path = f'{poses_folder}/{file_name.replace(".jpg", "_keypoints.pt")}' # img_100041_keypoints.pt
    
    keypoints = torch.load(keypoints_file_path) # A numpy array with shape [1, 1, 17, 3] 
                                                # representing the keypoint coordinates and scores
                                                # returned from the MoveNet model.
    disp_img = tf.image.resize_with_pad(orig_img, pose_config.INPUT_SIZE, pose_config.INPUT_SIZE)
    # disp_img = tf.expand_dims(disp_img, axis=0)
    # display_images(axes, orig_img, annotated_img, pose_img, pose_cropped_img)
    ax_row[0].imshow(annotated_img)
    
    display_keypoints(ax_row, disp_img, pose_img, pose_cropped_img, keypoints, colors, shapes)
    
    # fig.tight_layout(pad=0)
    # ax.margins(0)
    # ax.set_yticklabels([])
    # ax.set_xticklabels([])
    # plt.axis('off')
         
feature_helpers.FeatureExtractor(config, face_config, pose_config, tqdm).keypoints_relative_to_nose()
# normalize_poses()


  0%|          | 0/6000 [00:00<?, ?it/s]

,filename,label,pose_feature_vector,nose_offset_x,left_eye_offset_x,right_eye_offset_x,left_ear_offset_x,right_ear_offset_x,left_shoulder_offset_x,right_shoulder_offset_x,...,left_elbow_score,right_elbow_score,left_wrist_score,right_wrist_score,left_hip_score,right_hip_score,left_knee_score,right_knee_score,left_ankle_score,right_ankle_score
0,img_48336.jpg,4,"[-0.035036057, -0.035036057, -0.018196791, -0....",0.332170,-0.035036,-0.018197,-0.011111,0.033738,0.063421,0.215807,...,0.559622,0.599662,0.438087,0.759579,0.571074,0.564975,0.190885,0.082692,0.090743,0.134912
1,img_86426.jpg,4,"[-0.029750407, -0.029750407, -0.020493448, -0....",0.324019,-0.029750,-0.020493,0.000614,0.023370,0.079093,0.225625,...,0.276838,0.517142,0.395247,0.632883,0.467937,0.378535,0.207561,0.315662,0.049092,0.109102
2,img_38238.jpg,2,"[-0.029954404, -0.029954404, -0.0071044564, -0...",0.385913,-0.029954,-0.007104,-0.011733,0.031726,0.047659,0.180604,...,0.752064,0.870633,0.731195,0.731981,0.587788,0.360084,0.349760,0.244974,0.077215,0.085454
3,img_28681.jpg,4,"[-0.027178586, -0.027178586, -0.015425503, -0....",0.321195,-0.027179,-0.015426,0.017147,0.048965,0.100663,0.270724,...,0.246459,0.691087,0.479120,0.740284,0.403327,0.551018,0.343422,0.722791,0.337236,0.081227
4,img_80378.jpg,1,"[-0.029777497, -0.029777497, -0.020891935, -0....",0.305588,-0.029777,-0.020892,-0.001573,0.011584,0.051158,0.202056,...,0.412501,0.862169,0.247147,0.783414,0.631854,0.479554,0.056887,0.204816,0.098185,0.063777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,img_59929.jpg,5,"[-0.041088223, -0.041088223, -0.022706777, -0....",0.309577,-0.041088,-0.022707,-0.032507,0.007465,0.031562,0.186271,...,0.639355,0.281787,0.769919,0.113592,0.541353,0.622801,0.087762,0.339461,0.117406,0.069263
5996,img_58743.jpg,4,"[-0.034503043, -0.034503043, -0.021009296, -0....",0.312540,-0.034503,-0.021009,-0.022420,0.012607,0.096042,0.201875,...,0.208693,0.615836,0.400202,0.617838,0.368065,0.377731,0.223935,0.330053,0.083124,0.181887
5997,img_46864.jpg,6,"[-0.030128628, -0.030128628, -0.007858992, -0....",0.335278,-0.030129,-0.007859,0.009149,0.065909,0.087384,0.206149,...,0.457411,0.617456,0.512566,0.598345,0.673614,0.708538,0.328401,0.653334,0.120384,0.104205
5998,img_39068.jpg,8,"[-0.008832008, -0.008832008, 0.016371638, 0.01...",0.297652,-0.008832,0.016372,0.069449,0.111182,0.110459,0.284387,...,0.469213,0.728690,0.752742,0.643261,0.341937,0.309014,0.294337,0.732635,0.172147,0.023645


## Angle Made By Each Hand.


In [238]:
# Idea is to find the angle that the line connecting elbow and shoulder makes with 
# the line connecting elbow and wrist.
import math
 
def find_angles():
    def getAnglePy(params):
        a0, a1, b0, b1, c0, c1 = params
        ang = math.degrees(math.atan2(c1-b1, c0-b0) - math.atan2(a1-b1, a0-b0))
        ang = ang + 360 if ang < 0 else ang
        return ang
    keypoints_df = feature_helpers.FeatureExtractor(config, face_config, pose_config, tqdm).load_keypoints(count=5)
    projected_df = keypoints_df[['left_shoulder_x', 'left_shoulder_y', 'left_elbow_x', 'left_elbow_y', 'left_wrist_x', 'left_wrist_y']]
    keypoints_df['left_hand_angle'] = projected_df.apply(getAnglePy, axis=1)
    projected_df = keypoints_df[['right_shoulder_x', 'right_shoulder_y', 'right_elbow_x', 'right_elbow_y', 'right_wrist_x', 'right_wrist_y']]    
    keypoints_df['right_hand_angle'] = projected_df.apply(getAnglePy, axis=1)
    print(keypoints_df)
find_angles()

  0%|          | 0/6000 [00:00<?, ?it/s]

   label       filename
0      7  img_45037.jpg
1      2  img_73202.jpg
2      4   img_7466.jpg
3      4  img_98481.jpg
4      8  img_89028.jpg
5      8  img_82672.jpg
201.10691716930026
197.63420451914786
248.887268980701
335.6561616932093
224.54057687293016
215.19491381502058
262.88458556398666
330.89706812619005
253.35527881481025
248.7487907058516
208.4119640987503
315.6666831364142
        filename  label    nose_x  left_eye_x  right_eye_x  left_ear_x  \
0  img_45037.jpg      7  0.354819    0.304450     0.344531    0.303446   
1  img_73202.jpg      2  0.320999    0.291201     0.317308    0.321574   
2   img_7466.jpg      4  0.304032    0.275121     0.295372    0.319340   
3  img_98481.jpg      4  0.363299    0.340871     0.345609    0.365115   
4  img_89028.jpg      8  0.302083    0.273047     0.281602    0.316494   
5  img_82672.jpg      8  0.352352    0.316518     0.348416    0.342890   

   right_ear_x  left_shoulder_x  right_shoulder_x  left_elbow_x  ...  \
0     0.401937     